In [ ]:
# Cell 1 — cài đặt
!pip install -q bing-image-downloader gradio tensorflow pillow matplotlib


In [ ]:
# Cell 2 — mount Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Cell 3 — cấu hình lớp và mô tả (bạn có thể sửa text mô tả cho phù hợp)
classes = {
    "life_long": "Đường sinh đạo dài (LifeLine_Long):\n\n- Thường được giải thích là người có sức sống và thể lực bền bỉ, tinh thần ổn định.\n- Người có đường sinh đạo dài thường được cho là có sức đề kháng tốt, ít ốm yếu, thích nghi nhanh.\n- (Ghi chú: đây là mô tả tham khảo giải trí, không phải chẩn đoán y tế.)",
    "life_short": "Đường sinh đạo ngắn (LifeLine_Short):\n\n- Thường được gán ý là người có thể cần chú ý hơn đến sức khỏe, đôi khi cuộc sống có nhiều thay đổi.\n- Không có nghĩa là số phận ngắn; chỉ là ký hiệu hình dạng đường sinh đôi khi ngắn so với tiêu chuẩn.",
    "head_strong": "Đường trí tuệ rõ / mạnh (HeadLine_Strong):\n\n- Biểu thị tư duy sắc bén, logic, khả năng tập trung tốt.\n- Người có đường trí đạo rõ nét được mô tả là quyết đoán, suy nghĩ thấu đáo.",
    "heart_clear": "Đường tâm rõ ràng (HeartLine_Clear):\n\n- Thể hiện sự minh bạch trong cảm xúc, khả năng biểu đạt tình cảm rõ ràng.\n- Có thể biểu thị sự cân bằng cảm xúc, dễ thân thiết với người khác.",
    "mixed": "Kiểu hỗn hợp / khó phân loại (Mixed):\n\n- Ảnh thuộc nhóm này có thể chứa nhiều yếu tố lẫn lộn (nhiều đường nhỏ, góc chụp khác nhau), hoặc không rõ rệt về một đường cụ thể.\n- Kết quả dự đoán cho nhóm này mang tính “không chắc” và dùng để hiển thị thông tin tham khảo."
}

# đặt tên thư mục dataset
dataset_dir = "palm_dataset"   # nơi lưu ảnh tải về


In [ ]:
# Cell 4 — tải ảnh (chạy 1 lần)
import os
from bing_image_downloader import downloader

if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir, exist_ok=True)
    for label in classes.keys():
        query = f"palm lines {label} hand palm"  # query hơi generic để lấy nhiều kiểu ảnh
        out_dir = os.path.join(dataset_dir, label)
        print("Downloading for", label)
        downloader.download(query, limit=80, output_dir=dataset_dir, adult_filter_off=True, force_replace=False, timeout=60)
    print("✅ Tải xong (hoặc đã lưu).")
else:
    print("Dataset đã tồn tại, bỏ qua bước tải.")


[%] Downloading Images to /content/palm_dataset/palm lines life_long hand palm


[!!]Indexing page: 1

[%] Indexed 35 Images on Page 1.


[%] Downloading Image #1 from https://i.pinimg.com/originals/4f/72/34/4f7234d47c9afca3562fbb0c70a56368.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://i.pinimg.com/originals/59/3b/e0/593be0969f11cf023e560ab3b345d613.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://theredheadriter.com/wp-content/uploads/2013/01/Read-Palm-Life-Line-Palmistry.jpg
[%] File Downloaded !

[%] Downloading Image #4 from https://images.chinahighlights.com/allpicture/2021/08/0855924323bd44aea9f757f3_748x467.jpg
[%] File Downloaded !

[%] Downloading Image #5 from https://destinypalmistry.com/wp-content/uploads/2019/07/life-line-timescale.jpg
[%] File Downloaded !

[%] Downloading Image #6 from https://chinaler.com/wp-content/uploads/2023/01/life-line.jpg
[%] File Downloaded !

[%] Downloading Image #7 from https://i.pinimg.com/736x/9d/51/58/9d5

In [ ]:
# Cell 5 — chuẩn hóa tên thư mục con
import shutil, glob

# tìm thư mục con do bing tạo và map về tên label nếu cần
subdirs = [d for d in glob.glob(dataset_dir + "/*") if os.path.isdir(d)]
print("Thư mục con hiện có:", subdirs)

# nếu bing tạo folder giống query, ta sẽ match bằng chứa tên label
for d in subdirs:
    name = os.path.basename(d).lower()
    for label in classes.keys():
        if label.replace("_"," ") in name or label in name:
            # move files into desired folder
            target = os.path.join(dataset_dir, label)
            if not os.path.exists(target):
                os.makedirs(target, exist_ok=True)
            # move images
            for f in os.listdir(d):
                src = os.path.join(d, f)
                dst = os.path.join(target, f)
                try:
                    shutil.move(src, dst)
                except:
                    pass
            # remove empty dir
            try:
                os.rmdir(d)
            except:
                pass

# nếu còn folder không đúng tên, in ra để bạn kiểm tra
print("Sau chuẩn hoá, thư mục con:", [d for d in os.listdir(dataset_dir)])


Thư mục con hiện có: ['palm_dataset/palm lines life_long hand palm', 'palm_dataset/palm lines life_short hand palm', 'palm_dataset/palm lines heart_clear hand palm', 'palm_dataset/palm lines mixed hand palm', 'palm_dataset/palm lines head_strong hand palm']
Sau chuẩn hoá, thư mục con: ['life_short', 'mixed', 'life_long', 'head_strong', 'heart_clear']


In [ ]:
# Cell 6 — dọn ảnh corrupted, quá nhỏ và in số lượng
from PIL import Image

min_wh = 40
removed = 0
for label in classes.keys():
    folder = os.path.join(dataset_dir, label)
    if not os.path.exists(folder):
        print("Warning: folder missing:", folder)
        continue
    files = os.listdir(folder)
    for fn in files:
        fp = os.path.join(folder, fn)
        try:
            im = Image.open(fp)
            im.verify()  # kiểm tra ảnh bị lỗi
            im = Image.open(fp)
            w,h = im.size
            if w < min_wh or h < min_wh:
                os.remove(fp); removed += 1
        except:
            try:
                os.remove(fp); removed += 1
            except:
                pass
print(f"Đã xóa {removed} file hỏng/quá nhỏ.")

# in thống kê
for label in classes.keys():
    folder = os.path.join(dataset_dir, label)
    cnt = len(os.listdir(folder)) if os.path.exists(folder) else 0
    print(f"{label}: {cnt} ảnh")


Đã xóa 0 file hỏng/quá nhỏ.
life_long: 80 ảnh
life_short: 80 ảnh
head_strong: 80 ảnh
heart_clear: 80 ảnh
mixed: 80 ảnh


In [ ]:
# Cell 7 — tạo train/test bằng ImageDataGenerator (tự chia bằng validation_split)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (128,128)
BATCH = 16

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.12,
    height_shift_range=0.12,
    shear_range=0.12,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split=0.18
)

train_gen = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="categorical",
    subset="training",
    shuffle=True
)

val_gen = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="categorical",
    subset="validation",
    shuffle=False
)

# lưu labels vào drive để load sau này
import pickle
labels = list(train_gen.class_indices.keys())
with open("/content/drive/MyDrive/palm_labels.pkl", "wb") as f:
    pickle.dump(labels, f)
print("Classes:", labels)


Found 317 images belonging to 5 classes.
Found 67 images belonging to 5 classes.
Classes: ['head_strong', 'heart_clear', 'life_long', 'life_short', 'mixed']


In [ ]:
# Cell 8 — xây ANN và train
from tensorflow.keras import layers, models, callbacks
import tensorflow as tf

num_classes = len(labels)

model = models.Sequential([
    layers.Flatten(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.Dense(1024, activation="relu"),
    layers.Dropout(0.4),
    layers.Dense(512, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(num_classes, activation="softmax")
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss="categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

# callbacks
chk = callbacks.ModelCheckpoint("/content/drive/MyDrive/palm_model_best.h5", save_best_only=True, monitor="val_accuracy", mode="max")
es = callbacks.EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True)

history = model.fit(train_gen, validation_data=val_gen, epochs=30, callbacks=[chk, es])
print("✅ Train hoàn thành. Model tốt nhất đã lưu lên Drive.")


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 49152)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │    50,332,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,990,085 (194.51 MB)

 Trainable params: 50,990,085 (194.51 MB)

...

Epoch 1/30
 3/20 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.2465 - loss: 3.2741

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2024 - loss: 5.2285

20/20 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.2023 - loss: 5.2617 - val_accuracy: 0.2090 - val_loss: 2.3996
Epoch 2/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2196 - loss: 6.4858

20/20 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - accuracy: 0.2202 - loss: 6.4490 - val_accuracy: 0.2239 - val_loss: 2.0190
Epoch 3/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.2011 - loss: 4.1073 - val_accuracy: 0.2239 - val_loss: 1.7769
Epoch 4/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.2075 - loss: 3.2571 - val_accuracy: 0.1791 - val_loss: 2.1993
Epoch 5/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.2142 - loss: 2.4951 - val_accuracy: 0.1940 - val_loss: 1.7265
Epoch 6/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.1785 - loss: 1.9727 - val_accuracy: 0.1940 - val_loss: 1.6498
Epoch 7/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.2184 - loss: 1.6570 - val_accuracy: 0.2090 - val_loss: 1.6100
Epoch 8/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.1571 - loss: 1.6082 - val_accuracy: 0.1791 - val_loss: 1.6099
Epoch 9/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.1641 - loss: 1.6133 - val_accuracy: 0.1940 - val_loss: 1.6101
Epo

In [ ]:
# Cell 9 — load nhanh khi mở lại Colab
from tensorflow.keras.models import load_model
import pickle

model = load_model("/content/drive/MyDrive/palm_model_best.h5")
with open("/content/drive/MyDrive/palm_labels.pkl", "rb") as f:
    labels = pickle.load(f)

print("✅ Đã load model và labels:", labels)


✅ Đã load model và labels: ['head_strong', 'heart_clear', 'life_long', 'life_short', 'mixed']


In [ ]:
# Cell 10 — hàm dự đoán trả top-3 + mô tả dài
import numpy as np
from PIL import Image

def predict_palm(img_pil):
    img = img_pil.resize(IMG_SIZE)
    x = np.array(img) / 255.0
    x = np.expand_dims(x, 0)
    preds = model.predict(x)[0]
    idxs = np.argsort(preds)[-3:][::-1]
    results = []
    for i in idxs:
        results.append({"label": labels[i], "score": float(preds[i]), "desc": classes[labels[i]]})
    return results


In [ ]:
# Cell 11 — lưu feedback và update model function
import os
feedback_dir = "/content/drive/MyDrive/palm_feedback"
os.makedirs(feedback_dir, exist_ok=True)

def save_feedback(img_pil, correct_label):
    if correct_label not in labels:
        return f"Nhãn không hợp lệ. Hợp lệ: {labels}"
    label_dir = os.path.join(feedback_dir, correct_label)
    os.makedirs(label_dir, exist_ok=True)
    cnt = len(os.listdir(label_dir))
    path = os.path.join(label_dir, f"fb_{cnt}.jpg")
    img_pil.save(path)
    return f"Đã lưu feedback vào {path}"

def update_from_feedback():
    # fine-tune model trên feedback nếu có
    gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    if not os.path.exists(feedback_dir):
        return "Chưa có feedback."
    # ensure folders exist
    sub = [d for d in os.listdir(feedback_dir) if os.path.isdir(os.path.join(feedback_dir,d))]
    if len(sub) == 0:
        return "Chưa có feedback."
    fb_flow = gen.flow_from_directory(feedback_dir, target_size=IMG_SIZE, batch_size=8, class_mode="categorical")
    if fb_flow.samples == 0:
        return "Chưa có feedback."
    model.fit(fb_flow, epochs=3)
    model.save("/content/drive/MyDrive/palm_model_best.h5")
    return f"Đã cập nhật model từ feedback ({fb_flow.samples} ảnh)."


In [ ]:
# Cell 12 — Web Gradio
import gradio as gr

def gr_predict(img):
    res = predict_palm(img)
    # hiển thị top1 text + bottom top3 probabilities
    top = res[0]
    summary = f"**Dự đoán chính:** {top['label']} — độ tin cậy {top['score']*100:.1f}%\n\n{top['desc']}"
    probs = {r['label']: round(r['score'], 4) for r in res}
    return summary, probs

def gr_feedback(img, correct_label):
    return save_feedback(img, correct_label)

def gr_update():
    return update_from_feedback()

with gr.Blocks() as demo:
    gr.Markdown("# 🖐️ PalmLines AI — Phân tích chỉ tay (demo giải trí)")
    with gr.Row():
        with gr.Column(scale=2):
            inp = gr.Image(type="pil", label="Tải ảnh lòng bàn tay lên (nền sáng, rõ đường)")
            btn = gr.Button("Dự đoán")
            out_text = gr.Markdown()
        with gr.Column(scale=1):
            gr.Markdown("### Top-3 dự đoán")
            out_label = gr.Label(num_top_classes=3)
            gr.Markdown("### Nếu sai, nhập nhãn đúng:")
            correct = gr.Dropdown(choices=labels, label="Chọn nhãn đúng (feedback)")
            fb_btn = gr.Button("Lưu feedback")
            fb_msg = gr.Textbox(label="Thông báo")
            upd_btn = gr.Button("Cập nhật mô hình từ feedback")
            upd_msg = gr.Textbox(label="Kết quả cập nhật")

    btn.click(gr_predict, inputs=inp, outputs=[out_text, out_label])
    fb_btn.click(gr_feedback, inputs=[inp, correct], outputs=fb_msg)
    upd_btn.click(gr_update, outputs=upd_msg)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4f6c753a258814657a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
